In [1]:
from transformers import AutoProcessor, AutoModelForCTC
processor = AutoProcessor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-telugu-tem-100")
model = AutoModelForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-telugu-tem-100")

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import pandas as pd
import os
from IPython import display
import datasets
from datasets.info import DatasetInfosDict
from datasets import load_dataset,concatenate_datasets, DatasetDict, Dataset, Audio
import torch

In [3]:
fleurs_dataset = load_dataset("google/fleurs", "te_in", split="test",num_proc=32)
fleurs_dataset = fleurs_dataset.rename_column("transcription", "text")
fleurs_dataset = fleurs_dataset.cast_column("audio",Audio(sampling_rate=16000))
fleurs_dataset = fleurs_dataset.remove_columns(['id', 'num_samples', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])
fleurs_dataset

Found cached dataset fleurs (/raid/cs20mds14030/.cache/huggingface/datasets/google___fleurs/te_in/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


Dataset({
    features: ['path', 'audio', 'text'],
    num_rows: 472
})

In [4]:
sampling_rate = fleurs_dataset.features["audio"].sampling_rate

In [30]:
def transcribe(audio_array):
    inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe speech
    transcription = processor.batch_decode(predicted_ids)
    transcription = transcription[0].replace("<s>","")
    return transcription

In [54]:
fleurs_dataset['audio'][0]

{'path': '7038975664927295261.wav',
 'array': array([0.        , 0.        , 0.        , ..., 0.00052094, 0.00167996,
        0.00159758]),
 'sampling_rate': 16000}

In [60]:
from tqdm import tqdm
vakyansh_texts = []
for item in tqdm(fleurs_dataset['audio']):
    vakyansh_texts.append(transcribe(item['array']))
vakyansh_texts

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.04it/s]


['రెండవ సెట్లో డెల్పోట్రో్ ఆధిక్యం లభించినా కూడా ఈ సెట్లో కూడా ఆరు ఆర్మించి ఆరుకి చేరుకున్న తర్వాత డైబ్రేక్ అని వార్యం అయ్యింది ',
 'హాజరైన వారి సంఖ్య ఎంత ఎక్కువ అంటే సైన్పీటర్ స్క్వేర్ లో జరిగిన అంత్క్రియలను చూడడానికి అందరికీ సాధ్యపడలేదు ']

In [62]:
import evaluate
metric = evaluate.load("wer")
vakyansh_wer = 100 * metric.compute(predictions=vakyansh_texts, references=fleurs_dataset['text'])
print(vakyansh_wer)

53.333333333333336
